In [604]:
import pandas as pd
import time
from datetime import datetime, timedelta
import json
import requests
import re
import numpy as np
import matplotlib as plt
from bs4 import BeautifulSoup

In [605]:
salaryHtml = requests.get('https://www.spotrac.com/nfl/contracts/cumulative-cash//')
bs = BeautifulSoup(salaryHtml.content, 'html.parser')
time.sleep(1)

#links for players
playersRaw = bs.find_all("td",{"class":"player"})
playersLinks = []
for td in playersRaw:
    playersLinks.append(td.find("a")["href"])

updatedPlayersLinks = playersLinks[:len(playersLinks) - 300]

In [606]:
PlayerList = []
PositionList = []
AgeList = []
TeamList = []
ExperienceList = [] 
DraftedDetailsList = []
CollegeList = []
AgentList = []
SalaryList = []
PlayerStatusList = []

PlayerDetailesRaw1 = []
PlayerDetailesRaw2 = []


1


In [599]:
for href in updatedPlayersLinks:
    CashearningHtml = requests.get(href + "cash-earnings/")
    bs = BeautifulSoup(CashearningHtml.content, 'html.parser')
    
    PlayerDetailesRaw1 = bs.find("div",{"class":"player-option medium"})
    PlayerDetailesRaw2 = bs.find("div",{"class":"player-option large"})
    
    #Player name
    try:
        PlayerList.append(bs.find("h1").text.strip())
    except:
        PlayerList.append(np.nan)
    
    #player position and team
    try:
        PlayerPos = PlayerDetailesRaw1.find("span",{"class":"player-item position"}).text.split(", ")
        TeamList.append(PlayerPos[0])
        PositionList.append(PlayerPos[1])
    except:
        TeamList.append(np.nan)
        PositionList.append(np.nan)
    
    #player age and experience  
    try:
        PlayerAgeExp = PlayerDetailesRaw1.find("span",{"class":"player-infoitem"})
        #searching text if exist
        if PlayerDetailesRaw1.find(text="Age:"):
            AgeList.append(PlayerAgeExp.text)
            PlayerAgeExp = PlayerAgeExp.findNext("span",{"class":"player-infoitem"})
        else:
            AgeList.append(np.nan)
        #searching text if exist
        if PlayerDetailesRaw1.find(text="Exp: "):
            ExperienceList.append(PlayerAgeExp.text)
        else:
            ExperienceList.append(np.nan)
    except:
        AgeList.append(np.nan)
        ExperienceList.append(np.nan)
    
     #player drafte details, college and agent  
    try:
        Player = PlayerDetailesRaw2.find("span",{"class":"player-infoitem"})
        if PlayerDetailesRaw2.find(text="Drafted:"):
            DraftedDetailsList.append(Player.text)
            Player = Player.findNext("span",{"class":"player-infoitem"})
        else:
            DraftedDetailsList.append(np.nan)
    
        if PlayerDetailesRaw2.find(text="College:"):
            CollegeList.append(Player.text)
            Player = Player.findNext("span",{"class":"player-infoitem"})
        else:
            CollegeList.append(np.nan)

        if PlayerDetailesRaw2.find(text="Agent(s):"):
            AgentList.append(Player.text)
        else:
            AgentList.append(np.nan)

    except:
        DraftedDetailsList.append(np.nan)
        CollegeList.append(np.nan)
        AgentList.append(np.nan)

    #player salary
    earnings = bs.find("table",{"class":"earningstable rtable"}).find("tbody").findAll("td",{"class":"right visible-lg"})
    if earnings:
        SalaryList.append(earnings[0].text)
    else:
        SalaryList.append(np.nan)
    
    #player status
    try:
        PlayerStatusList.append(bs.find("div",{"class":"player-status"}).text.strip())
    except:
        PlayerStatusList.append(np.nan)
        
    time.sleep(1)
    
df = pd.DataFrame({"Player Name":PlayerList,"Salary":SalaryList,"Experience Years":ExperienceList,"Team":TeamList,"Position":PositionList,"Age":AgeList,"Drafted Details":DraftedDetailsList,"College":CollegeList,"Agent":AgentList,"Player Status":PlayerStatusList})
df

,Player Name,Salary,Experience Years,Team,Position,Age,Drafted Details,College,Agent,Player Status
0,Dak Prescott,"$450,000",5 Years,Cowboys,QB,27-24d,"Round 4 (#135 overall), 2016",Mississippi State,Todd France (CAA Sports),Active
1,Aaron Rodgers,"$230,000",16 Years,Packers,QB,37-20d,"Round 1 (#24 overall), 2005",California,David Dunn (Athletes First),Active
2,Matthew Stafford,"$3,100,000",12 Years,Rams,QB,33-15d,"Round 1 (#1 overall), 2009",Georgia,Tom Condon (CAA Sports),Active
3,Joey Bosa,"$450,000",5 Years,Chargers,DE,25-11d,"Round 1 (#3 overall), 2016",Ohio State,Brian Ayrault (CAA Sports),Active
4,Jimmy Garoppolo,"$420,000",7 Years,49ers,QB,29-20d,"Round 2 (#62 overall), 2014",Eastern Illinois,Don Yee (Yee & Dubin Sports),Active
5,Khalil Mack,"$420,000",7 Years,Bears,OLB,30-0d,"Round 1 (#5 overall), 2014",Buffalo,Joel Segal (Lagardere Unlimited),Active
6,Aaron Donald,"$420,000",7 Years,Rams,DT,30-30d,"Round 1 (#13 overall), 2014",Pittsburgh,Brian Ayrault (CAA Sports),Active
7,Kirk Cousins,"$390,000",9 Years,Vikings,QB,32-3d,"Round 4 (#102 overall), 2012",Michigan State,Priority Sports & Entertainment,Active
8,Ryan Tannehill,"$390,000",9 Years,Titans,QB,32-26d,"Round 1 (#8 overall), 2012",Texas A&M,Brian Ayrault (CAA Sports),Active
9,Eli Manning,"$1,744,000",15 Years,Giants,QB,40-19d,"Round 1 (#1 overall), 2004",Mississippi,CAA Sports (Tom Condon),Inactive


In [583]:
df_try.to_csv("NFLPlayerDetails.csv",index= False)

In [600]:
df_try = pd.read_csv("NFLPlayerDetails.csv")
df_try

,Player Name,Salary,Experience Years,Team,Position,Age,Drafted Details,College,Agent,Player Status
0,Dak Prescott,"$450,000",5 Years,Cowboys,QB,27-24d,"Round 4 (#135 overall), 2016",Mississippi State,Todd France (CAA Sports),Active
1,Aaron Rodgers,"$230,000",16 Years,Packers,QB,37-20d,"Round 1 (#24 overall), 2005",California,David Dunn (Athletes First),Active
2,Matthew Stafford,"$3,100,000",12 Years,Rams,QB,33-15d,"Round 1 (#1 overall), 2009",Georgia,Tom Condon (CAA Sports),Active
3,Joey Bosa,"$450,000",5 Years,Chargers,DE,25-11d,"Round 1 (#3 overall), 2016",Ohio State,Brian Ayrault (CAA Sports),Active
4,Jimmy Garoppolo,"$420,000",7 Years,49ers,QB,29-20d,"Round 2 (#62 overall), 2014",Eastern Illinois,Don Yee (Yee & Dubin Sports),Active
...,...,...,...,...,...,...,...,...,...,...
4267,Colby Gossett,-,2 Years,Browns,G,25-29d,"Round 6 (#213 overall), 2018",Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active
4268,Jeremiah Valoaga,"$300,883",3 Years,Raiders,DE,26-7d,NaN,"2017, Lions",Nevada-Las Vegas,Inactive
4269,Hunter Sharp,"$23,750",1 Years,Bengals,WR,27-27d,NaN,"2016, Eagles",Utah State,Inactive
4270,LaTroy Lewis,"$86,400",1 Years,Titans,ILB,27-13d,NaN,"2017,",Tennessee,Inactive
